In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import vartools as vt

In [22]:
equities = pd.read_excel('Datos_Examen_Market_Risk.xlsx')
equities['Date'] = pd.to_datetime(equities['Date'])
equities = equities.set_index('Date')
equities.head()

,ABT,KXI,QQQ,X,XOM
Date,,,,,
2020-01-02,79.6899,49.6289,209.9761,10.5066,55.6312
2020-01-03,78.7185,49.5841,208.0528,10.4289,55.1839
2020-01-06,79.1309,49.7366,209.3932,10.4775,55.6076
2020-01-07,78.6910,49.4136,209.3641,10.8076,55.1525
2020-01-08,79.0117,49.4046,210.9378,10.7785,54.3208


In [23]:
def var_stocks(data, positions, nc, long):
    rend = data.pct_change().dropna()
    port_value = (data.iloc[-1] * positions).sum()
    w = (data*positions).iloc[-1] / port_value
    port_rend = np.dot(w, rend.T)
    
    if long == True:
        var_p = np.percentile(port_rend, 100-nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    else:
        var_p = np.percentile(port_rend, nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend > var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
        
    resultados = pd.DataFrame({
        '':['%', '$'],
        'VaR':[var_p, var_m],
        'C-VaR':[es_p, es_m]
        })
    return resultados

In [34]:
positions = [5432, 8972, 3243, 7456, 5931]
long = True
nc = 99
resultados = var_stocks(equities, positions, nc, long)
resultados

,,VaR,C-VaR
0,%,-0.0371,-0.0553
1,$,"-141,504.6683","-210,988.8385"


In [6]:
w_objetivo = [0.1625, 0.4406, 0.2104, 0.0075, 0.179]

In [25]:
def resumen_rebalanceo(data, w_objetivo, positions):
    w = (data.iloc[-1] * positions) / (data.iloc[-1] * positions).sum()
    port_value = (data.iloc[-1] * positions).sum() 
    resumen = pd.DataFrame({
    'Pesos Originales': w,
    'Pesos Objetivo': w_objetivo,
    'Comprar o Vender': np.floor((w_objetivo-w) * port_value / data.iloc[-1])
    })
    return resumen

resumen = resumen_rebalanceo(equities, w_objetivo, positions)
resumen

,Pesos Originales,Pesos Objetivo,Comprar o Vender
ABT,0.1643,0.1625,-60.0000
KXI,0.1497,0.4406,"17,435.0000"
QQQ,0.4421,0.2104,"-1,700.0000"
X,0.0688,0.0075,"-6,644.0000"
XOM,0.1750,0.1790,135.0000


In [8]:
divisas = pd.read_excel('Datos_Examen_Market_Risk.xlsx', sheet_name='Data_FX')
divisas = divisas[['Date','JPYUSD', 'MXNUSD']]
divisas['Date'] = pd.to_datetime(divisas['Date'])
divisas = divisas.set_index('Date')
divisas.head()

,JPYUSD,MXNUSD
Date,,
2020-01-01,0.0092,0.0529
2020-01-02,0.0092,0.0529
2020-01-03,0.0092,0.0531
2020-01-06,0.0093,0.0528
2020-01-07,0.0092,0.0531


In [9]:
def var_es(data, positions, nc, long):
    rend_inversion = ((data * positions).sum(axis=1)).pct_change().dropna()
    if long == True:
        percentil =  (1-nc)*100
        var_p = np.percentile(rend_inversion, percentil)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = rend_inversion[rend_inversion < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    else:
        percentil = nc*100
        var_p = np.percentile(rend_inversion, percentil)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = rend_inversion[rend_inversion > var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    resultados = pd.DataFrame({'VAR %':var_p, 'VAR $':var_m, 'ES %':es_p, 'ES $':es_m}, index=['Resultados'])
    return resultados

In [19]:
positions = [73000000, 49000000]
long = False
nc = 0.99

var_es(divisas, positions, nc, long)

,VAR %,VAR $,ES %,ES $
Resultados,0.0158,"46,019.0110",0.0192,"55,955.3110"


In [16]:
46019

46019

In [20]:
call_blsdelta = lambda s, k, r, sigma, T: norm.cdf( ( np.log(s/k) + (r + sigma**2 / 2) * T ) / ( sigma * np.sqrt(T)) )
put_blsdelta = lambda s, k, r, sigma, T: np.abs(norm.cdf( ( np.log(s/k) + (r + sigma**2 / 2) * T ) / ( sigma * np.sqrt(T)) ) - 1)

def delta_hedge(calls, puts, noc_c, noc_p):
    return np.array([call_blsdelta(*i) for i in calls]).dot(noc_c) - np.array([put_blsdelta(*i) for i in puts]).dot(noc_p)

In [21]:
#                  Spot, Strike, r,     sigma, T
calls = np.array([ [2694, 2650, 0.043916, 0.31, 3/12],    #C1
                   [2694, 2670, 0.043916, 0.28, 3/12],    #C2
                   [2694, 2650, 0.042142, 0.43, 12/12],   #C3
                   [2694, 2670, 0.042142, 0.37, 12/12]])  #C4

 #                  Spot, Strike, r,     sigma, T
puts =  np.array([ [2694, 2730, 0.043916, 0.35, 3/12],    #C1
                   [2694, 2740, 0.043916, 0.36, 3/12],    #C2
                   [2694, 2730, 0.042142, 0.51, 12/12],   #C3
                   [2694, 2740, 0.042142, 0.58, 12/12]])  #C4

#                 C1, C2, C3, C4
noc_c = np.array([21, 25, 28, 33])

#                 P1, P2, P3, P4
noc_p = np.array([45, 10, 12, 43])

net_1 = delta_hedge(calls, puts, noc_c, noc_p)

if net_1 < 0:
    print(f'Hay que vender ${net_1:.4f} MDD del subyacente')
else:
    print(f'Hay que comprar ${net_1:.4f} MDD del subyacente')

Hay que comprar $19.3827 MDD del subyacente
